# **NETT MARGIN Optimization**

In [1]:
pip install --upgrade scipy

Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (1.6.3)


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

##**Data Preparation**

In [3]:
df = pd.read_excel('/content/drive/MyDrive/Optimization Plant.xlsx')
df.head()

,Case,DEMAND,KOTA,JENIS_ITEM,Unnamed: 4,GUDANG,BIAYA HANDLING @ CARTON,ONGKIR-BDG @ CARTON,ONGKIR-YOGYA @ CARTON,ONGKIR-MLG @ CARTON,ONGKIR-JKT @ CARTON,INDEX_MARGIN AA,INDEX_MARGIN BB,INDEX_MARGIN_CC,Capacity AA,Capacity BB,Capacity CC,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,20000.0,BANDUNG,AA,NaN,PLANT1,1000,2500,1500.0,2000.0,2000.0,50000.0,45000.0,60000.0,15000.0,20000.0,12000.0,NaN,NaN,NaN
1,2,12000.0,BANDUNG,BB,NaN,PLANT2,1100,2400,1000.0,2000.0,2500.0,50000.0,50000.0,70000.0,22000.0,15000.0,15000.0,NaN,NaN,NaN
2,3,12000.0,BANDUNG,CC,NaN,PLANT3,1200,1000,3000.0,2000.0,3000.0,60000.0,45000.0,55000.0,20000.0,10000.0,27000.0,NaN,NaN,NaN
3,4,15000.0,YOGYA,AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,11000.0,YOGYA,BB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
arr_df = df.to_numpy()
print(arr_df)

[[1 20000.0 'BANDUNG' 'AA' nan 'PLANT1' 1000 2500 1500.0 2000.0 2000.0
  50000.0 45000.0 60000.0 15000.0 20000.0 12000.0 nan nan nan]
 [2 12000.0 'BANDUNG' 'BB' nan 'PLANT2' 1100 2400 1000.0 2000.0 2500.0
  50000.0 50000.0 70000.0 22000.0 15000.0 15000.0 nan nan nan]
 [3 12000.0 'BANDUNG' 'CC' nan 'PLANT3' 1200 1000 3000.0 2000.0 3000.0
  60000.0 45000.0 55000.0 20000.0 10000.0 27000.0 nan nan nan]
 [4 15000.0 'YOGYA' 'AA' nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan]
 [5 11000.0 'YOGYA' 'BB' nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan]
 [6 10500.0 'YOGYA' 'CC' nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan]
 [7 15000.0 'MALANG' 'AA' nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan]
 [8 8500.0 'MALANG' 'BB' nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan]
 [9 9000.0 'MALANG' 'CC' nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan]
 [10 12000.0 'JAKARTA' 'AA' nan nan nan nan n

In [5]:
demand = arr_df[0:12,1:4]
demand = demand[np.argsort(demand[:,2])]
demand = np.reshape(demand,(3,4,-1))
print(demand)

[[[20000.0 'BANDUNG' 'AA']
  [15000.0 'YOGYA' 'AA']
  [15000.0 'MALANG' 'AA']
  [12000.0 'JAKARTA' 'AA']]

 [[12000.0 'BANDUNG' 'BB']
  [11000.0 'YOGYA' 'BB']
  [8500.0 'MALANG' 'BB']
  [7000.0 'JAKARTA' 'BB']]

 [[12000.0 'BANDUNG' 'CC']
  [10500.0 'YOGYA' 'CC']
  [9000.0 'MALANG' 'CC']
  [10000.0 'JAKARTA' 'CC']]]


In [6]:
a = np.unique(demand[:,:,2])
b = np.unique(demand[:,:,1])
num_product = a.shape[0]
num_dest = b.shape[0]

In [7]:
plant = arr_df[0:3,6:-3]
num_plant = 3
print(plant)

[[1000 2500 1500.0 2000.0 2000.0 50000.0 45000.0 60000.0 15000.0 20000.0
  12000.0]
 [1100 2400 1000.0 2000.0 2500.0 50000.0 50000.0 70000.0 22000.0 15000.0
  15000.0]
 [1200 1000 3000.0 2000.0 3000.0 60000.0 45000.0 55000.0 20000.0 10000.0
  27000.0]]


In [8]:
# make array that contain plant nett based on product and region
plant_nett_region = np.zeros(((num_product,num_dest,num_plant)))
for i in range(0,num_product):
  for j in range(0,num_dest):
    for k in range(0,num_plant):
      plant_nett_region[i,j,k] = plant[k,i+5]-plant[k,j+1]-plant[k,0]
print(plant_nett_region)

[[[46500. 46500. 57800.]
  [47500. 47900. 55800.]
  [47000. 46900. 56800.]
  [47000. 46400. 55800.]]

 [[41500. 46500. 42800.]
  [42500. 47900. 40800.]
  [42000. 46900. 41800.]
  [42000. 46400. 40800.]]

 [[56500. 66500. 52800.]
  [57500. 67900. 50800.]
  [57000. 66900. 51800.]
  [57000. 66400. 50800.]]]


In [9]:
capacity_plant = plant[0:plant.shape[0],-3:plant.shape[1]]
capacity_plant = np.transpose(capacity_plant)
print(capacity_plant)

[[15000.0 22000.0 20000.0]
 [20000.0 15000.0 10000.0]
 [12000.0 15000.0 27000.0]]


## **Optimization**

In [10]:
copy_capacity_plant = np.array(capacity_plant)
copy_plant_nett_region = np.array(plant_nett_region)
copy_demand = np.array(demand)
plant_opt = np.zeros(((num_product,num_dest,num_plant)))
num = plant_nett_region.shape[1]*plant_nett_region.shape[2]

In [11]:
def linprogam(mthd):
  total = 0
  balance = []
  for i in range(0,num_plant):
    c = copy_plant_nett_region[i].transpose()
    c = c.reshape(-1)
    c *= -1
    A = np.zeros((copy_plant_nett_region.shape[-1]+copy_plant_nett_region.shape[-2], copy_plant_nett_region.shape[-1]*copy_plant_nett_region.shape[-2]))
    B = []
    l = 0
    u = 0
    for n in range(0,A.shape[0]):
      if n < copy_plant_nett_region.shape[-1]:
        for k in range(0,copy_plant_nett_region.shape[-2]):
            A[n,l] = 1
            l = l+1
      else:
        m = 0
        for k in range(0,copy_plant_nett_region.shape[-1]):
          A[n,u+m] = 1
          m = m + 4
        u = u + 1
      B.append(A[n])
    b = [copy_capacity_plant[i,0]]
    for n in range(1,copy_capacity_plant.shape[1]):
      b.append(copy_capacity_plant[i,n])
    for n in range(0,copy_demand.shape[1]):
      b.append(copy_demand[i,n,0])
    sim = linprog(c, A_ub=B, b_ub=b, method=mthd)
    total = total + sim.fun*-1
    balance.append(sim.x)
  return sim,total,mthd,balance

In [12]:
s = linprogam('simplex')
r = linprogam('revised simplex')
hi = linprogam('highs-ipm')
hs =linprogam('highs')
hd = linprogam('highs-ds')
lst = [s,r,hi,hs,hd]
for i in lst:
  print('Method (', i[2],') \n Total Nett Margin:', i[1],'\n')

Method ( simplex ) 
 Total Nett Margin: 7077600000.0 

Method ( revised simplex ) 
 Total Nett Margin: 7077600000.0 

Method ( highs-ipm ) 
 Total Nett Margin: 7077600000.0 

Method ( highs ) 
 Total Nett Margin: 7077600000.0 

Method ( highs-ds ) 
 Total Nett Margin: 7077600000.0 



In [13]:
b = np.array(s[3]).reshape(copy_plant_nett_region.shape[0],copy_plant_nett_region.shape[2],copy_plant_nett_region.shape[1])
optimum = np.zeros(((copy_plant_nett_region.shape[0],copy_plant_nett_region.shape[1],copy_plant_nett_region.shape[2])))
for i in range(0,copy_plant_nett_region.shape[0]):
  for j in range(0,copy_plant_nett_region.shape[1]):
    for k in range(0,copy_plant_nett_region.shape[2]):
      optimum[i,j,k] = b[i,k,j]

In [14]:
optimum

array([[[    0.,     0., 20000.],
        [    0., 15000.,     0.],
        [ 3000.,  7000.,     0.],
        [12000.,     0.,     0.]],

       [[    0.,  2000., 10000.],
        [    0., 11000.,     0.],
        [ 6500.,  2000.,     0.],
        [ 7000.,     0.,     0.]],

       [[    0.,     0., 12000.],
        [    0., 10500.,     0.],
        [ 2000.,  4500.,  2500.],
        [10000.,     0.,     0.]]])



## **Result**



In [15]:
a = optimum.reshape(12,-1)
b = demand.reshape(12,-1)
c = plant_nett_region.reshape(12,-1)
total = a * c

In [16]:
all = np.concatenate((b,a,c,total),axis=1)

In [17]:
finish = pd.DataFrame(all, columns = ['Demand','Kota','Produk','Plant 1','Plant 2','Plant 3','Nett Margin 1','Nett Margin 2','Nett Margin 3','Total 1','Total 2','Total 3'])

In [18]:
finish

,Demand,Kota,Produk,Plant 1,Plant 2,Plant 3,Nett Margin 1,Nett Margin 2,Nett Margin 3,Total 1,Total 2,Total 3
0,20000,BANDUNG,AA,0,0,20000,46500,46500,57800,0,0,1.156e+09
1,15000,YOGYA,AA,0,15000,0,47500,47900,55800,0,7.185e+08,0
2,15000,MALANG,AA,3000,7000,0,47000,46900,56800,1.41e+08,3.283e+08,0
3,12000,JAKARTA,AA,12000,0,0,47000,46400,55800,5.64e+08,0,0
4,12000,BANDUNG,BB,0,2000,10000,41500,46500,42800,0,9.3e+07,4.28e+08
5,11000,YOGYA,BB,0,11000,0,42500,47900,40800,0,5.269e+08,0
6,8500,MALANG,BB,6500,2000,0,42000,46900,41800,2.73e+08,9.38e+07,0
7,7000,JAKARTA,BB,7000,0,0,42000,46400,40800,2.94e+08,0,0
8,12000,BANDUNG,CC,0,0,12000,56500,66500,52800,0,0,6.336e+08
9,10500,YOGYA,CC,0,10500,0,57500,67900,50800,0,7.1295e+08,0


In [19]:
finish.to_excel('/content/drive/MyDrive/test1b-edwinsanjaya.xlsx', header=True, index=False)